In [1]:
import pandas as pd
import os
import sqlite3
from sqlite3 import Error

In [2]:
from constants import * 
from utils import *

In [3]:
print(DB_PATH)
print(DB_FILE_NAME)

/home/airflow/dags/Assignment/02_training_pipeline/
lead_scoring_data_cleaning.db


In [4]:
cnx = sqlite3.connect(DB_PATH + DB_FILE_NAME)
df = pd.read_sql(f"SELECT name FROM sqlite_master WHERE type='table'", cnx)
print(df.shape)
df.head()

(7, 1)


name
0            target
1  predicted_output
2          features
3       loaded_data
4  city_tier_mapped

In [5]:
df = pd.read_sql('select * from interactions_mapped', cnx)
print(df.shape)
df.head()

(24885, 13)


index         created_date  city_tier first_platform_c first_utm_medium_c  \
0      0  2021-07-01 01:38:17        3.0           Level3             Level0   
1      1  2021-07-01 02:41:36        1.0           Level0             others   
2      2  2021-07-01 03:25:23        1.0           Level3             Level0   
3      3  2021-07-01 03:43:23        3.0           Level3             Level0   
4      4  2021-07-01 03:43:29        1.0           Level3             Level0   

  first_utm_source_c  total_leads_droppped  referred_lead  \
0             Level0                   2.0            0.0   
1             others                   1.0            0.0   
2             others                   3.0            0.0   
3             others                   2.0            0.0   
4             others                   1.0            0.0   

   assistance_interaction  career_interaction  payment_interaction  \
0                     0.0                 0.0                  0.0   
1                     0.0                 0.0                  0.0   
2                     0.0                 0.0                  0.0   
3                     0.0                 0.0                  0.0   
4                     0.0                 0.0                  0.0   

   social_interaction  syllabus_interaction  
0                 0.0                   0.0  
1                 0.0                   0.0  
2                 0.0                   0.0  
3                 0.0                   0.0  
4                 0.0                   0.0

In [6]:
encoded_df = df[FEATURES_TO_ENCODE]
encoded_df.head()

first_platform_c first_utm_medium_c first_utm_source_c
0           Level3             Level0             Level0
1           Level0             others             others
2           Level3             Level0             others
3           Level3             Level0             others
4           Level3             Level0             others

In [7]:
columns = df.columns.to_list()
print(columns)
print(len(columns))

['index', 'created_date', 'city_tier', 'first_platform_c', 'first_utm_medium_c', 'first_utm_source_c', 'total_leads_droppped', 'referred_lead', 'assistance_interaction', 'career_interaction', 'payment_interaction', 'social_interaction', 'syllabus_interaction']
13


In [8]:
%time
df_new_encoded = pd.get_dummies(df, columns = FEATURES_TO_ENCODE,drop_first=True)

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 8.58 µs


In [9]:
columns = df_new_encoded.columns.to_list()
print(columns)
print(len(columns))

['index', 'created_date', 'city_tier', 'total_leads_droppped', 'referred_lead', 'assistance_interaction', 'career_interaction', 'payment_interaction', 'social_interaction', 'syllabus_interaction', 'first_platform_c_Level1', 'first_platform_c_Level2', 'first_platform_c_Level3', 'first_platform_c_Level7', 'first_platform_c_Level8', 'first_platform_c_others', 'first_utm_medium_c_Level1', 'first_utm_medium_c_Level2', 'first_utm_medium_c_Level3', 'first_utm_medium_c_Level7', 'first_utm_medium_c_Level8', 'first_utm_medium_c_others', 'first_utm_source_c_Level1', 'first_utm_source_c_Level2', 'first_utm_source_c_Level3', 'first_utm_source_c_Level7', 'first_utm_source_c_others']
27


In [10]:
db_path=DB_PATH
db_file_name=DB_FILE_NAME
one_hot_encoded_features=ONE_HOT_ENCODED_FEATURES
features_to_encode=FEATURES_TO_ENCODE
model_config=MODEL_CONFIG
experiment=EXPERIMENT
tracking_uri=TRACKING_URI

In [11]:
FEATURES_TO_ENCODE

['first_platform_c', 'first_utm_medium_c', 'first_utm_source_c']

In [12]:
encode_features(db_path, db_file_name, one_hot_encoded_features, features_to_encode)

Data has been extracted successfully from lead_scoring_model_experimentation.


KeyError: "['app_complete_flag'] not found in axis"

In [ ]:
## Verify data written to table

In [13]:
df = pd.read_sql(f"SELECT name FROM sqlite_master WHERE type='table'", cnx)
print(df.shape)
df.head()

(7, 1)


name
0            target
1  predicted_output
2          features
3       loaded_data
4  city_tier_mapped

In [14]:
cnx = sqlite3.connect(db_path + db_file_name)
X = pd.read_sql('select * from features', cnx)
X.head()

index  first_platform_c  first_utm_medium_c  first_utm_source_c  \
0      0                 0                   0                   0   
1      1                 0                   0                   0   
2      2                 0                   0                   0   
3      3                 0                   0                   0   
4      4                 0                   0                   0   

   total_leads_droppped  city_tier  first_platform_c_Level8  \
0                   2.0        3.0                        0   
1                   1.0        1.0                        0   
2                   3.0        1.0                        0   
3                   2.0        3.0                        0   
4                   1.0        1.0                        0   

   first_platform_c_others  first_platform_c_Level2  \
0                        0                        0   
1                        0                        0   
2                        0                        0   
3                        0                        0   
4                        0                        0   

   first_utm_medium_c_others  first_utm_medium_c_Level0  \
0                          0                          1   
1                          1                          0   
2                          0                          1   
3                          0                          1   
4                          0                          1   

   first_platform_c_Level7  first_platform_c_Level0  
0                        0                        0  
1                        0                        1  
2                        0                        0  
3                        0                        0  
4                        0                        0

In [15]:
y = pd.read_sql('select * from target', cnx)
y.head()

index  app_complete_flag
0      0                  1
1      1                  1
2      2                  1
3      3                  0
4      4                  0

## create sql lite connection and start database

In [16]:
# create a connection and setup a SQLite database with the name "lead_scoring_model_experimentation.db" in 
# 'Assignment/02_training_pipeline/notebook/' location
def create_sqlit_connection(db_path, db_file):
    """ create a database connection to a SQLite database """
    conn = None
    # opening the conncetion for creating the sqlite db
    try:
        conn = sqlite3.connect(db_path + db_file)
        print(sqlite3.version)
    # return an error if connection not established
    except Error as e:
        print(e)
    # closing the connection once the database is created
    finally:
        if conn:
            conn.close()

In [18]:
create_sqlit_connection(DB_PATH,r"Lead_scoring_mlflow_production.db")

2.6.0


In [ ]:
#mlflow server --backend-store-uri='sqlite:////home/Assignment/02_training_pipeline/scripts/database/Lead_scoring_mlflow_production.db' --default-artifact-root="/home/Assignment/02_training_pipeline/scripts/mlruns/" --port=6006 --host=0.0.0.0

In [19]:
mlflow.set_tracking_uri("http://0.0.0.0:6006")

In [21]:
get_trained_model(db_path, db_file_name, model_config, experiment,tracking_uri)

ValueError: Found input variables with inconsistent numbers of samples: [24885, 238124]